In [1]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import math
import gc

In [2]:
# ! cp data/Train_target.csv ./raw_data/Train_target.csv
# ! cp data/Test_target.csv ./raw_data/Test_target.csv

In [3]:
train=pd.read_csv("./data/Train.csv")
test=pd.read_csv("./data/Test.csv")
sample_sub=pd.read_csv("./data/sample_sub.csv")

In [4]:
train.head()


,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press,target
0,ID_train_0,C,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...",45.126304
1,ID_train_1,D,"22.53333333,21.71666667,20.83333333,20.9833333...","0.102,0.0,0.0,0.0,0.0,0.0,0.0,0.034,0.017,0.01...","0.744583333,0.808083333,0.911166667,0.91633333...","281.6643101,89.15629262,81.96853891,291.018632...","2.3775,1.126666667,0.700833333,0.3416666670000...","90.32,90.3775,90.44083333,90.4725,90.45416667,...",79.131702
2,ID_train_10,A,"28.975,27.95,29.6,26.425,22.09166667,21.775,22...","0.0,0.0,0.0,0.102,0.136,0.0,0.0,2.16,1.276,0.0...","0.573333333,0.597166667,0.5668333329999999,0.6...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","88.55166667,88.46416667,88.31916667,88.24,88.2...",32.661304
3,ID_train_100,A,"22.96666667,24.26666667,25.275,25.625,25.86666...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.77,3.012,1.0...","0.8430833329999999,0.79025,0.7375,0.728,0.7049...","300.0850574,293.6769595,294.5174647,301.921416...","1.446666667,1.1925,1.324166667,1.5441666669999...","88.615,88.53083333,88.4,88.27166667,88.2075,88...",53.850238
4,ID_train_1000,A,"21.875,21.575,21.525,21.43333333,20.50833333,1...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.8564166670000001,0.874916667,0.879833333,0.8...","21.83997432,17.05405341,89.26406044,123.585424...","0.1975,0.244166667,0.411666667,0.56,0.5775,0.4...","88.55666667,88.64083333,88.65833333,88.6475,88...",177.418750


In [5]:
type(train.precip[2])

str

In [6]:
test.head()

,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press
0,ID_test_0,C,"23.96666667,22.88333333,22.28333333,21.4833333...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.6898333329999999,0.722333333,0.7755833329999...","290.1352379,328.47011289999995,323.5730796,282...","0.6333333329999999,0.725,0.955,0.8725,0.6925,0...","87.64333333,87.67,87.70583333,87.75083333,87.7..."
1,ID_test_1,C,"18.99166667,19.70833333,20.95833333,22.7833333...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.99975,0.99725,0.956333333,0.851583333,0.7775...","295.2350543,307.6569135,313.7360236,311.518385...","0.640833333,1.244166667,1.284166667,1.2875,1.3...","87.8925,87.95666667,87.9925,87.9925,87.9808333..."
2,ID_test_10,D,"25.11666667,25.08333333,24.40833333,23.575,22....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.760166667,0.733583333,0.78475,0.825833333,0....","77.06161145,95.48124997,61.44498258,89.4688360...","2.0133333330000003,1.3025,0.783333333,0.530833...","90.34,90.2975,90.305,90.35583333,90.4425,90.49..."
3,ID_test_100,C,"18.71666667,20.60833333,22.28333333,23.9833333...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.604,0.0,0.0,...","0.951416667,0.92575,0.8603333329999999,0.77883...","310.7509058,316.186793,324.8167326,327.2174992...","0.745,1.169166667,1.316666667,1.385,1.40083333...","87.98833333,88.06333333,88.07833333,88.0366666..."
4,ID_test_1000,D,"17.63333333,18.05833333,19.625,21.28333333,23....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.9,0.898333333,0.8725,0.831666667,0.820833332...","334.6054069,329.6848561000001,322.7668036,312....","0.620833333,0.5,0.456666667,0.558333333,1.0158...","90.515,90.5575,90.62333333,90.6825,90.67916667..."


In [7]:
# covert features  from string to List of values 
def replace_nan(x):
    if x==" ":
        return np.nan
    else :
        return float(x)
        
features=["temp","precip","rel_humidity","wind_dir","wind_spd","atmos_press"]
for feature in features : 
    train[feature]=train[feature].apply(lambda x: [ replace_nan(X) for X in x.replace("nan"," ").split(",")])
    test[feature]=test[feature].apply(lambda x: [ replace_nan(X)  for X in x.replace("nan"," ").split(",")])    

In [8]:
#train.precip[2]

### Features engineering part

In [9]:
def aggregate_features(x,col_name):
    #x["max_"+col_name]=x[col_name].apply(np.max)
    #x["min_"+col_name]=x[col_name].apply(np.min)
    x["mean_"+col_name]=x[col_name].apply(np.mean) #average
    #x["std_"+col_name]=x[col_name].apply(np.std)
    #x["var_"+col_name]=x[col_name].apply(np.var)
    #x["median_"+col_name]=x[col_name].apply(np.median) # separe higher from lower values
    #x["ptp_"+col_name]=x[col_name].apply(np.ptp) # range of values peak to peak max -min
    return x  


def remove_nan_values(x):
    return [e for e in x if not math.isnan(e)]


In [10]:
data=pd.concat([train,test],sort=False).reset_index(drop=True)
data.columns.tolist()

['ID',
 'location',
 'temp',
 'precip',
 'rel_humidity',
 'wind_dir',
 'wind_spd',
 'atmos_press',
 'target']

In [11]:
test.shape

(5035, 8)

In [12]:
len(data.temp[0])

121

In [13]:
for col_name in tqdm(features):
    data[col_name]=data[col_name].apply(remove_nan_values)

100%|██████████| 6/6 [00:01<00:00,  5.68it/s]


In [14]:
len(data.temp[0])

16

In [15]:
data

,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press,target
0,ID_train_0,C,"[26.90909091, 27.20833333, 26.18333333, 24.7, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.5108181820000001, 0.535416667, 0.6145, 0.63...","[272.90275160000004, 104.565241, 167.1772247, ...","[0.800909091, 1.073333333, 1.5175, 1.4625, 0.5...","[87.77727273, 87.6525, 87.61416667, 87.6375, 8...",45.126304
1,ID_train_1,D,"[22.53333333, 21.71666667, 20.83333333, 20.983...","[0.102, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.034, 0...","[0.744583333, 0.808083333, 0.911166667, 0.9163...","[281.6643101, 89.15629262, 81.96853891, 291.01...","[2.3775, 1.126666667, 0.700833333, 0.341666667...","[90.32, 90.3775, 90.44083333, 90.4725, 90.4541...",79.131702
2,ID_train_10,A,"[28.975, 27.95, 29.6, 26.425, 22.09166667, 21....","[0.0, 0.0, 0.0, 0.102, 0.136, 0.0, 0.0, 2.16, ...","[0.573333333, 0.597166667, 0.5668333329999999,...","[328.68291380000005, 307.8251459, 319.0177512,...","[1.0325, 1.193333333, 1.275833333, 1.493333333...","[88.55166667, 88.46416667, 88.31916667, 88.24,...",32.661304
3,ID_train_100,A,"[22.96666667, 24.26666667, 25.275, 25.625, 25....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.77,...","[0.8430833329999999, 0.79025, 0.7375, 0.728, 0...","[300.0850574, 293.6769595, 294.5174647, 301.92...","[1.446666667, 1.1925, 1.324166667, 1.544166666...","[88.615, 88.53083333, 88.4, 88.27166667, 88.20...",53.850238
4,ID_train_1000,A,"[21.875, 21.575, 21.525, 21.43333333, 20.50833...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.8564166670000001, 0.874916667, 0.879833333,...","[21.83997432, 17.05405341, 89.26406044, 123.58...","[0.1975, 0.244166667, 0.411666667, 0.56, 0.577...","[88.55666667, 88.64083333, 88.65833333, 88.647...",177.418750
...,...,...,...,...,...,...,...,...,...
20569,ID_test_995,C,"[25.925, 25.80833333, 24.58333333, 24.0, 24.05...","[0.0, 0.0, 0.0, 0.408, 0.017, 0.0, 0.0, 0.0, 0...","[0.645416667, 0.637416667, 0.705333333, 0.6988...","[329.8319536, 343.2603743, 167.5577726, 161.14...","[0.749166667, 0.7425, 1.3766666669999998, 1.49...","[87.8075, 87.73833333, 87.66166667, 87.5566666...",NaN
20570,ID_test_996,D,"[22.16666667, 21.68333333, 20.15833333, 19.825...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.82275, 0.827666667, 0.876416667, 0.88891666...","[291.1252201000001, 277.1125974, 248.1977595, ...","[0.980833333, 0.78, 0.45, 0.46, 0.425833333, 0...","[90.75166667, 90.68916667, 90.61916667, 90.57,...",NaN
20571,ID_test_997,E,"[17.35833333, 18.4, 22.125, 25.40833333, 25.96...","[0.034, 0.017, 0.017, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.875833333, 0.898333333, 0.8666666670000001,...","[18.92517827, 2.478476814, 337.74147410000006,...","[1.4391666669999998, 1.466666667, 1.63, 1.9325...","[88.47666667, 88.535, 88.61416667, 88.6625, 88...",NaN
20572,ID_test_998,A,"[24.16666667, 24.59166667, 18.01818182, 21.708...","[0.0, 0.0, 7.519, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[0.831666667, 0.8091666670000001, 0.968181818,...","[313.0287989, 308.591496, 209.3230512, 151.182...","[1.706666667, 1.979166667, 0.999166667, 0.5416...","[88.61166667, 88.5325, 88.505, 88.40166667, 88...",NaN


In [16]:
for col_name in tqdm(features):
    data=aggregate_features(data,col_name)

100%|██████████| 6/6 [00:00<00:00,  6.98it/s]


In [17]:
data.shape

(20574, 15)

In [18]:
data.head()

,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press,target,mean_temp,mean_precip,mean_rel_humidity,mean_wind_dir,mean_wind_spd,mean_atmos_press
0,ID_train_0,C,"[26.90909091, 27.20833333, 26.18333333, 24.7, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.5108181820000001, 0.535416667, 0.6145, 0.63...","[272.90275160000004, 104.565241, 167.1772247, ...","[0.800909091, 1.073333333, 1.5175, 1.4625, 0.5...","[87.77727273, 87.6525, 87.61416667, 87.6375, 8...",45.126304,22.299527,0.000000,0.758530,209.574655,0.677348,87.752434
1,ID_train_1,D,"[22.53333333, 21.71666667, 20.83333333, 20.983...","[0.102, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.034, 0...","[0.744583333, 0.808083333, 0.911166667, 0.9163...","[281.6643101, 89.15629262, 81.96853891, 291.01...","[2.3775, 1.126666667, 0.700833333, 0.341666667...","[90.32, 90.3775, 90.44083333, 90.4725, 90.4541...",79.131702,24.679063,0.007025,0.758058,177.109855,0.899208,90.429924
2,ID_train_10,A,"[28.975, 27.95, 29.6, 26.425, 22.09166667, 21....","[0.0, 0.0, 0.0, 0.102, 0.136, 0.0, 0.0, 2.16, ...","[0.573333333, 0.597166667, 0.5668333329999999,...","[328.68291380000005, 307.8251459, 319.0177512,...","[1.0325, 1.193333333, 1.275833333, 1.493333333...","[88.55166667, 88.46416667, 88.31916667, 88.24,...",32.661304,24.112317,0.151767,0.712944,201.373211,1.038656,88.420931
3,ID_train_100,A,"[22.96666667, 24.26666667, 25.275, 25.625, 25....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.77,...","[0.8430833329999999, 0.79025, 0.7375, 0.728, 0...","[300.0850574, 293.6769595, 294.5174647, 301.92...","[1.446666667, 1.1925, 1.324166667, 1.544166666...","[88.615, 88.53083333, 88.4, 88.27166667, 88.20...",53.850238,20.845273,1.127273,0.898326,259.973977,1.365202,88.376915
4,ID_train_1000,A,"[21.875, 21.575, 21.525, 21.43333333, 20.50833...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.8564166670000001, 0.874916667, 0.879833333,...","[21.83997432, 17.05405341, 89.26406044, 123.58...","[0.1975, 0.244166667, 0.411666667, 0.56, 0.577...","[88.55666667, 88.64083333, 88.65833333, 88.647...",177.418750,21.870732,0.005100,0.785134,181.944779,0.729763,88.538104


In [19]:
#train_mean=data[data.target.notnull()].reset_index(drop=True)
#test51=data[data.target.isna()].reset_index(drop=True)

In [20]:
data.to_csv('data/data_15_mean.csv')

In [21]:
"""data.to_pickle('data/data_51.pkl')
train51.to_pickle('data/train_51.pkl')
test51.to_pickle('data/test_51.pkl')"""



"data.to_pickle('data/data_51.pkl')\ntrain51.to_pickle('data/train_51.pkl')\ntest51.to_pickle('data/test_51.pkl')"

In [22]:
"""data.to_csv('data/data_51.csv')
train51.to_csv('data/train_51.csv')
test51.to_csv('data/test_51.csv')"""

"data.to_csv('data/data_51.csv')\ntrain51.to_csv('data/train_51.csv')\ntest51.to_csv('data/test_51.csv')"

In [23]:
"""for x in range(121): # 121 messdaten pro punkt 
    data["newtemp"+ str(x)] = data.temp.str[x]
    data["newprecip"+ str(x)] = data.precip.str[x]
    data["newrel_humidity"+ str(x)] = data.rel_humidity.str[x]
    data["newwind_dir"+ str(x)] = data.wind_dir.str[x]
    data["windspeed"+ str(x)] = data.wind_spd.str[x]
    data["atmospherepressure"+ str(x)] = data.atmos_press.str[x]"""

'for x in range(121): # 121 messdaten pro punkt \n    data["newtemp"+ str(x)] = data.temp.str[x]\n    data["newprecip"+ str(x)] = data.precip.str[x]\n    data["newrel_humidity"+ str(x)] = data.rel_humidity.str[x]\n    data["newwind_dir"+ str(x)] = data.wind_dir.str[x]\n    data["windspeed"+ str(x)] = data.wind_spd.str[x]\n    data["atmospherepressure"+ str(x)] = data.atmos_press.str[x]'

In [24]:
data.shape

(20574, 15)

In [25]:
features

['temp', 'precip', 'rel_humidity', 'wind_dir', 'wind_spd', 'atmos_press']

In [26]:
#data.drop(features,1,inplace=True)

In [27]:
data.shape

(20574, 15)

In [28]:
"""train=data[data.target.notnull()].reset_index(drop=True)
test=data[data.target.isna()].reset_index(drop=True)"""

'train=data[data.target.notnull()].reset_index(drop=True)\ntest=data[data.target.isna()].reset_index(drop=True)'

In [29]:
"""data.to_pickle('data/data_process.pkl')
train.to_pickle('data/train_process.pkl')
test.to_pickle('data/test_process.pkl')"""

"data.to_pickle('data/data_process.pkl')\ntrain.to_pickle('data/train_process.pkl')\ntest.to_pickle('data/test_process.pkl')"

In [30]:
"""data.to_csv('data/data_process.csv')
train.to_csv('data/train_process.csv')
test.to_csv('data/test_process.csv')"""

"data.to_csv('data/data_process.csv')\ntrain.to_csv('data/train_process.csv')\ntest.to_csv('data/test_process.csv')"

In [31]:
del data  
gc.collect()

96

In [32]:
train.head()

,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press,target
0,ID_train_0,C,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",45.126304
1,ID_train_1,D,"[22.53333333, 21.71666667, 20.83333333, 20.983...","[0.102, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.034, 0...","[0.744583333, 0.808083333, 0.911166667, 0.9163...","[281.6643101, 89.15629262, 81.96853891, 291.01...","[2.3775, 1.126666667, 0.700833333, 0.341666667...","[90.32, 90.3775, 90.44083333, 90.4725, 90.4541...",79.131702
2,ID_train_10,A,"[28.975, 27.95, 29.6, 26.425, 22.09166667, 21....","[0.0, 0.0, 0.0, 0.102, 0.136, 0.0, 0.0, 2.16, ...","[0.573333333, 0.597166667, 0.5668333329999999,...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[88.55166667, 88.46416667, 88.31916667, 88.24,...",32.661304
3,ID_train_100,A,"[22.96666667, 24.26666667, 25.275, 25.625, 25....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.77,...","[0.8430833329999999, 0.79025, 0.7375, 0.728, 0...","[300.0850574, 293.6769595, 294.5174647, 301.92...","[1.446666667, 1.1925, 1.324166667, 1.544166666...","[88.615, 88.53083333, 88.4, 88.27166667, 88.20...",53.850238
4,ID_train_1000,A,"[21.875, 21.575, 21.525, 21.43333333, 20.50833...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.8564166670000001, 0.874916667, 0.879833333,...","[21.83997432, 17.05405341, 89.26406044, 123.58...","[0.1975, 0.244166667, 0.411666667, 0.56, 0.577...","[88.55666667, 88.64083333, 88.65833333, 88.647...",177.418750


In [33]:
test.head()

,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press
0,ID_test_0,C,"[23.96666667, 22.88333333, 22.28333333, 21.483...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.6898333329999999, 0.722333333, 0.7755833329...","[290.1352379, 328.47011289999995, 323.5730796,...","[0.6333333329999999, 0.725, 0.955, 0.8725, 0.6...","[87.64333333, 87.67, 87.70583333, 87.75083333,..."
1,ID_test_1,C,"[18.99166667, 19.70833333, 20.95833333, 22.783...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.99975, 0.99725, 0.956333333, 0.851583333, 0...","[295.2350543, 307.6569135, 313.7360236, 311.51...","[0.640833333, 1.244166667, 1.284166667, 1.2875...","[87.8925, 87.95666667, 87.9925, 87.9925, 87.98..."
2,ID_test_10,D,"[25.11666667, 25.08333333, 24.40833333, 23.575...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.760166667, 0.733583333, 0.78475, 0.82583333...","[77.06161145, 95.48124997, 61.44498258, 89.468...","[2.0133333330000003, 1.3025, 0.783333333, 0.53...","[90.34, 90.2975, 90.305, 90.35583333, 90.4425,..."
3,ID_test_100,C,"[18.71666667, 20.60833333, 22.28333333, 23.983...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.604...","[0.951416667, 0.92575, 0.8603333329999999, 0.7...","[310.7509058, 316.186793, 324.8167326, 327.217...","[0.745, 1.169166667, 1.316666667, 1.385, 1.400...","[87.98833333, 88.06333333, 88.07833333, 88.036..."
4,ID_test_1000,D,"[17.63333333, 18.05833333, 19.625, 21.28333333...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.9, 0.898333333, 0.8725, 0.831666667, 0.8208...","[334.6054069, 329.6848561000001, 322.7668036, ...","[0.620833333, 0.5, 0.456666667, 0.558333333, 1...","[90.515, 90.5575, 90.62333333, 90.6825, 90.679..."


#### Cross validation strategy (Kfold,or simple train test split )

In [34]:
Experiment_name="simple_model"

In [35]:
import os
os.makedirs("proc_data", exist_ok=True)
os.makedirs("model_save/lgbm/{}".format(Experiment_name), exist_ok=True) # 
os.makedirs("model_save/catboost/{}".format(Experiment_name), exist_ok=True)
os.makedirs("model_save/xgboost/{}".format(Experiment_name), exist_ok=True)

In [36]:
try : 
    folds=pd.read_csv("./proc_data/folds_id.csv")
    train=train.merge(folds,on="ID",how="left")
    train.fold.nunique()
except : 
    #  you run this cell  only for the first time 
    from sklearn.model_selection import KFold 
    kfold=KFold(n_splits=5,shuffle=True,random_state=2020) # change this random_state or all of you will have the same score  :D 
    train.reset_index(drop=True,inplace=True)
    folds=train[["ID"]].copy()
    folds["fold"]=0
    for fold,(tr_indx,val_ind) in enumerate(kfold.split(folds)) : 
        folds.loc[val_ind,"fold"]=fold
    folds.to_csv("./proc_data/folds_id.csv",index=False)
    train=train.merge(folds,on="ID",how="left")
    
    del folds

### Model 

### Lgbm 

In [37]:
len(train.columns)

10

In [38]:
target_name="target"
id_name="ID"
features_to_remove=[target_name,id_name,"fold","location"]
features=train.columns.tolist()
features=[ fea for fea in  features if fea not in features_to_remove  ]


In [39]:
from sklearn.metrics import mean_squared_error
def metric(y,x):
    return np.sqrt(mean_squared_error(x,y))

In [41]:
import lightgbm as lgbm 
import xgboost as xgb 
#import catboost as cat 

In [42]:
def train_function(model,train,test,params,other_params,target_name,features,metric):
    folds_num=train.fold.nunique()
    validation=train[[id_name,"fold",target_name]].copy()
    validation["pred_"+target_name]=0
    sub=test[[id_name]].copy()
    sub[target_name]=0
    for fold in np.sort(train.fold.unique()):
        print("#"*50+" {} ".format(fold)+"#"*50)
        os.makedirs("model_save/lgbm/{}/{}".format(Experiment_name,str(int(fold))), exist_ok=True)
        X_train=train[train.fold!=fold]
        X_val=train[train.fold==fold]
        
        train_pred,validation_pred,test_pred=model(X_train,X_val,test,params,other_params)
        
        validation.loc[validation.fold==fold,"pred_"+target_name]=validation_pred
        sub[target_name]+=test_pred/folds_num
        train_score=metric(X_train[target_name],train_pred)
        val_score=metric(X_val[target_name],validation_pred)
        print("train score : {} validation score : {}".format(round(train_score,4),round(val_score,4)))
    final_validation_score=metric(validation[target_name],validation["pred_"+target_name])
    print("final validation score : {}".format(final_validation_score))
        
    return sub,validation,final_validation_score

def lgbm_model(X_train,X_val,X_test,params,other_params):
    dtrain = lgbm.Dataset(data=X_train[features], label=X_train[target_name], feature_name=features)
    dval = lgbm.Dataset(data=X_val[features], label=X_val[target_name], feature_name=features)

    model = lgbm.train(
        params=params,
        train_set=dtrain,
        num_boost_round=other_params["num_boost_round"],
        valid_sets=(dtrain, dval),
        early_stopping_rounds=other_params["early_stopping_rounds"],
        verbose_eval=other_params["verbose_eval"],
    )        
    best_iteration = model.best_iteration
    train_pred=model.predict(X_train[features], num_iteration=best_iteration)
    validation_pred=model.predict(X_val[features], num_iteration=best_iteration)
    test_pred=model.predict(test[features], num_iteration=best_iteration)
        
    return train_pred,validation_pred,test_pred

In [43]:
other_params={"num_boost_round":50000000,
              "early_stopping_rounds":50,
              "verbose_eval":1000,
}
lgbm_params = {
    "bagging_fraction": 0.8,
    "bagging_freq": 2,
    "boosting_type": "gbdt",
    "feature_fraction": 0.8,
    "learning_rate": 0.01,
    "max_depth": 8,
    "num_threads": 16,
    "objective": "regression",
    "metric": "rmse",
    "seed": 2020,
}

In [ ]:
"""sub,validation,score=train_function(model=lgbm_model,
                                    train=train,
                                    test=test,
                                    params=lgbm_params,
                                    other_params=other_params,
                                    target_name=target_name,
                                    features=features,
                                    metric=metric)"""

In [ ]:
#os.makedirs("subs", exist_ok=True)

In [ ]:
#sub.to_csv("./subs/lgbm_{}.csv".format(round(score,2)),index=False)